In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
# set up session
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [ ]:
# S3 path for training data
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [ ]:
!ls -ltr /opt/ml/model

In [ ]:
# approach 2 with TensorFlowModel class
# Model instance
sm_model = TensorFlowModel(
            model_data= model_s3_path, 
            framework_version=tf_framework_version,
            role=role) # IAM role of the notebook
# predictor
predictor = sm_model.deploy(initial_instance_count=1,
                            instance_type= instance_type)


In [ ]:
# deploying a Tensorflow/PyTorch/Other model files using EI
predictor = sm_model.deploy(
initial_instance_count=1, # ec2 initial count
   instance_type='ml.m4.xlarge', # ec2 instance type
   accelerator_type='ml.eia2.medium') # accelerator type
